In [19]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [20]:
def loadTrainData():
    cancer = load_breast_cancer()  # 加载乳腺癌数据
    X = cancer.data  # 加载乳腺癌判别特征
    y = cancer.target  # 两个TAG，y = 0时为阴性，y = 1时为阳性
    # 将数据集划分为训练集和测试集，测试集占比为0.2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = X_train.T
    X_test = X_test.T
    return X_train, X_test, y_train, y_test


In [21]:
def sigmoid(inx):
    from numpy import exp
    return 1.0/(1.0 + exp(-inx))

In [22]:
# 初始化参数
def initialize_para(dim):
    mu = 0
    sigma = 0.1
    np.random.seed(0)
    w = np.random.normal(mu, sigma, dim)
    w = np.reshape(w, (dim, 1))
    b = 0
    return w, b

In [23]:
# 前向传播
def propagate(w, b, X, Y):
    # eps防止log运算遇到0
    eps = 1e-5
    m = X.shape[1]
    # 计算初步运算结果
    A = sigmoid(np.dot(w.T, X) + b)
    # 计算损失函数值大小
    cost = -1 / m * np.sum(np.multiply(Y, np.log(A + eps)) +
                           np.multiply(1 - Y, np.log(1 - A + eps)))
    # 计算梯度值
    dw = 1 / m * np.dot(X, (A - Y).T)
    db = 1 / m * np.sum(A-Y)
    cost = np.squeeze(cost)

    grads = {"dw": dw,
             "db": db}
    # 返回损失函数大小以及反向传播的梯度值
    return grads, cost, A


In [24]:
# num_iterations 梯度下降次数
# learning_rate 学习率
def optimize(w, b, X, Y, num_iterations, learning_rate):
    costs = []  # 记录损失函数值

    # 循环进行梯度下降
    for i in range(num_iterations):
        # print(i)
        grads, cost, pre_Y = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]

        w = w - learning_rate * dw
        b = b - learning_rate * db

        # 每100次循环记录一次损失函数大小并打印
        if i % 100 == 0:
            costs.append(cost)

        if i % 100 == 0:
            pre_Y[pre_Y >= 0.5] = 1
            pre_Y[pre_Y < 0.5] = 0
            pre_Y = pre_Y.astype(np.int)
            acc = 1 - np.sum(pre_Y ^ Y) / len(Y)
            print("Iteration:{} Loss = {}, Acc = {}".format(i, cost, acc))

    # 最终参数值
    params = {"w": w,
              "b": b}

    return params, costs

In [25]:
def predict(w, b, X):
    # 样本个数
    m = X.shape[1]
    # 初始化预测输出
    Y_prediction = np.zeros((1, m))
    # 转置参数向量w
    w = w.reshape(X.shape[0], 1)

    # 预测结果
    Y_hat = sigmoid(np.dot(w.T, X) + b)

    # 将结果按照0.5的阈值转化为0/1
    for i in range(Y_hat.shape[1]):
        if Y_hat[:, i] > 0.5:
            Y_prediction[:, i] = 1
        else:
            Y_prediction[:, i] = 0

    return Y_prediction


初始化参数
对模型进行训练
在训练过程中进行前向传播，损失函数计算，梯度下降
找到最优解
测试集预测
模型评价


In [26]:
# 训练以及预测
def Logisticmodel(X_train, Y_train, X_test, Y_test, num_iterations=1000, learning_rate=0.1):
    # 初始化参数w，b
    w, b = initialize_para(X_train.shape[0])
    # 梯度下降找到最优参数
    parameters, costs = optimize(
        w, b, X_train, Y_train, num_iterations, learning_rate)

    w = parameters["w"]
    b = parameters["b"]

    # 训练集测试集的预测结果
    Y_prediction_train = predict(w, b, X_train)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_test = Y_prediction_test.T

    # 模型评价
    accuracy_score_value = accuracy_score(Y_test, Y_prediction_test)
    recall_score_value = recall_score(Y_test, Y_prediction_test)
    precision_score_value = precision_score(Y_test, Y_prediction_test)
    classification_report_value = classification_report(
        Y_test, Y_prediction_test)

    print("准确率:", accuracy_score_value)
    print("召回率:", recall_score_value)
    print("精确率:", precision_score_value)
    print(classification_report_value)

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test,
         "Y_prediction_train": Y_prediction_train,
         "w": w,
         "b": b,
         "learning_rate": learning_rate,
         "num_iterations": num_iterations}

    return d

In [27]:
if __name__ == '__main__':
    X_train, X_test, y_train, y_test = loadTrainData()
    Logisticmodel(X_train, y_train, X_test, y_test)


C:\Users\mingzi\AppData\Local\Temp/ipykernel_11280/2452862511.py:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pre_Y = pre_Y.astype(np.int)
C:\Users\mingzi\AppData\Local\Temp/ipykernel_11280/2408507428.py:3: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0 + exp(-inx))


Iteration:0 Loss = 4.149707398394874, Acc = 0.6395604395604395
Iteration:100 Loss = 7.261995447153545, Acc = 0.36923076923076925
Iteration:200 Loss = 1.2224219136749777, Acc = 0.8923076923076922
Iteration:300 Loss = 1.6699981769121266, Acc = 0.8549450549450549
Iteration:400 Loss = 1.1892593350937857, Acc = 0.8967032967032967
Iteration:500 Loss = 1.113328814240088, Acc = 0.9032967032967033
Iteration:600 Loss = 1.113328814240088, Acc = 0.9032967032967033
Iteration:700 Loss = 1.2434055249252671, Acc = 0.8901098901098901
Iteration:800 Loss = 1.0627225054242007, Acc = 0.9076923076923077
Iteration:900 Loss = 1.2398445885798706, Acc = 0.8923076923076922
准确率: 0.9210526315789473
召回率: 0.9090909090909091
精确率: 0.9523809523809523
              precision    recall  f1-score   support

           0       0.88      0.94      0.91        48
           1       0.95      0.91      0.93        66

    accuracy                           0.92       114
   macro avg       0.92      0.92      0.92       114
w